## Should contain
  * Whats pattern recog & detect?
  * Why deep learning?
  * Code

## Notes


  * ### [Image Classification](http://cs231n.github.io/classification/)
    * Image classification is the task of assigning an input image one label from a fixed set of categories
    * Challenges in Image classification
        * Viewpoint variation
        * Scale variation
        * Deformation
        * Occlusion
        * Illumination conditions
        * Background clutter
        * Intra-class variation